# 🌞 Example Weather App
> Using `prodb`


* Starting with a `.csv` of locations, we wish to call the [MetaWeather](https://www.metaweather.com/api/) API to pull in the weather forecast.  
* The app needs the ability to input new locations by the user

 🌍 Core functions will be converted into the streamlit app `weather_app.py`

In [ ]:
#hide
from nbdev.showdoc import *
import pandas as pd
%load_ext autoreload
%autoreload 2

## 1. Requests to `metaweather` open weather API

In [ ]:
import requests
import json
import arrow

In [ ]:
def get_json_from_query(location):
    """Search for a city and return metadata from API"""
    url = f"https://www.metaweather.com/api/location/search/?query={location}"
    r = requests.get(url).json()
    return r[0]

l = get_json_from_query('London')
l

{'title': 'London',
 'location_type': 'City',
 'woeid': 44418,
 'latt_long': '51.506321,-0.12714'}

Check `woeid`(Where On Earth ID) is correct for London

In [ ]:
assert l['woeid'] == 44418

Get weather data for the London `woeid`

In [ ]:
utc = arrow.utcnow().format('YYYY/MM/DD')
utc

'2021/10/22'

In [ ]:
def get_current_weather(location):
    """
    inputs: location str "London"
            to find woeid i.e. 44418
    """
    res = get_json_from_query(location)
    woeid = res['woeid']
    url = f"https://www.metaweather.com/api/location/{woeid}/"
    res = requests.get(url).json()
    return res

Today's forecast:

In [ ]:
res = get_current_weather('London')
res['consolidated_weather'][0]

{'id': 6455697949589504,
 'weather_state_name': 'Heavy Cloud',
 'weather_state_abbr': 'hc',
 'wind_direction_compass': 'W',
 'created': '2021-10-22T21:59:01.930990Z',
 'applicable_date': '2021-10-22',
 'min_temp': 6.88,
 'max_temp': 13.120000000000001,
 'the_temp': 12.215,
 'wind_speed': 8.008047157056126,
 'wind_direction': 271.66194481070727,
 'air_pressure': 1021.0,
 'humidity': 69,
 'visibility': 10.114369581643203,
 'predictability': 71}

In [ ]:
res

{'consolidated_weather': [{'id': 6455697949589504,
   'weather_state_name': 'Heavy Cloud',
   'weather_state_abbr': 'hc',
   'wind_direction_compass': 'W',
   'created': '2021-10-22T21:59:01.930990Z',
   'applicable_date': '2021-10-22',
   'min_temp': 6.88,
   'max_temp': 13.120000000000001,
   'the_temp': 12.215,
   'wind_speed': 8.008047157056126,
   'wind_direction': 271.66194481070727,
   'air_pressure': 1021.0,
   'humidity': 69,
   'visibility': 10.114369581643203,
   'predictability': 71},
  {'id': 5003891520307200,
   'weather_state_name': 'Heavy Cloud',
   'weather_state_abbr': 'hc',
   'wind_direction_compass': 'SSW',
   'created': '2021-10-22T21:59:01.778207Z',
   'applicable_date': '2021-10-23',
   'min_temp': 6.58,
   'max_temp': 14.870000000000001,
   'the_temp': 13.49,
   'wind_speed': 4.616571359397121,
   'wind_direction': 208.50083982409095,
   'air_pressure': 1025.5,
   'humidity': 68,
   'visibility': 10.523853197327607,
   'predictability': 71},
  {'id': 6605370312

There are a number of metrological properties available to us from the API. 

The ones we are most interested in are:
* `the_temp` 🌡️ current temperature
* `weather_state_name` ⛅ current sky condition
* `max_temp` 🥵 daily max temp
* `min_temp` 🥶 daily min temp


In [ ]:
res = get_current_weather('London')
df =  pd.DataFrame.from_records(res['consolidated_weather'][:1])
df['readable_time'] = df.created.apply(lambda x: arrow.get(x).humanize())
df

,id,weather_state_name,weather_state_abbr,wind_direction_compass,created,applicable_date,min_temp,max_temp,the_temp,wind_speed,wind_direction,air_pressure,humidity,visibility,predictability,readable_time
0,6455697949589504,Heavy Cloud,hc,W,2021-10-22T21:59:01.930990Z,2021-10-22,6.88,13.12,12.215,8.008047,271.661945,1021.0,69,10.11437,71,21 minutes ago


In [ ]:
def df_from_loc(location, 
                days_ahead=1,
                keep_cols='location the_temp readable_time created applicable_date local_time latt_long weather_state_name min_temp max_temp'.split(' ')):
    res = get_current_weather(location)
    df =  pd.DataFrame.from_records(res['consolidated_weather'][:days_ahead])
    df['location'] = location
    df['local_time'] = res['time']
    df['latt_long'] = res['latt_long']
    df['readable_time'] = df.created.apply(lambda x: arrow.get(x).humanize())
    return df[keep_cols]

df = df_from_loc('London', days_ahead=2)
df

,location,the_temp,readable_time,created,applicable_date,local_time,latt_long,weather_state_name,min_temp,max_temp
0,London,12.215,21 minutes ago,2021-10-22T21:59:01.930990Z,2021-10-22,2021-10-22T23:20:20.390435+01:00,"51.506321,-0.12714",Heavy Cloud,6.88,13.12
1,London,13.490,21 minutes ago,2021-10-22T21:59:01.778207Z,2021-10-23,2021-10-22T23:20:20.390435+01:00,"51.506321,-0.12714",Heavy Cloud,6.58,14.87


Example output showing location and current weather situation

In [ ]:
df = df_from_loc('London')
df

,location,the_temp,readable_time,created,applicable_date,local_time,latt_long,weather_state_name,min_temp,max_temp
0,London,12.215,21 minutes ago,2021-10-22T21:59:01.930990Z,2021-10-22,2021-10-22T23:20:22.091403+01:00,"51.506321,-0.12714",Heavy Cloud,6.88,13.12


In [ ]:
df = df_from_loc('Auckland')
df

,location,the_temp,readable_time,created,applicable_date,local_time,latt_long,weather_state_name,min_temp,max_temp
0,Auckland,15.615,2 hours ago,2021-10-22T19:37:05.363247Z,2021-10-23,2021-10-23T11:20:23.912248+13:00,"-36.884109,174.770416",Light Rain,12.795,15.555


In [ ]:
loc = get_json_from_query('Auckland')
# res = get_current_weather(l['woeid'])
# res[0]

In [ ]:
loc

{'title': 'Auckland',
 'location_type': 'City',
 'woeid': 2348079,
 'latt_long': '-36.884109,174.770416'}